In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import os
from google.colab import files
from sklearn.preprocessing import OneHotEncoder

In [7]:
fileList = os.listdir('/content/drive/MyDrive/prova/dades')
csv_files = [file for file in fileList if file[-3:] == "csv"]
batch_size = 1000
csv_files=sorted(csv_files)

# Crear una lista vacía para almacenar los DataFrames de cada archivo
dataframes = []

# Leer cada archivo CSV y agregarlo a la lista de DataFrames
for archivo in csv_files:

    df = pd.read_csv('/content/drive/MyDrive/prova/dades/'+archivo)
    df=df.loc[:, ['num_docks_available', 'Llocs','station_id','hour','dayofweek','month','Wind','Rain']]
    dataframes.append(df)

# Combinar todos los DataFrames en uno solo
dataframe_final = pd.concat(dataframes, ignore_index=True)

# Mostrar el DataFrame resultante
print(dataframe_final)

         num_docks_available  Llocs  station_id  hour  dayofweek  month  Wind  \
0                   5.500000   46.0           1   0.0          4    1.0     0   
1                   5.000000   46.0           1   1.0          4    1.0     0   
2                   5.000000   46.0           1   2.0          4    1.0     0   
3                   5.000000   46.0           1   3.0          4    1.0     0   
4                   5.000000   46.0           1   4.0          4    1.0     0   
...                      ...    ...         ...   ...        ...    ...   ...   
8819374            21.250000   22.0         519  18.0          4    9.0     0   
8819375            18.666667   22.0         519  19.0          4    9.0     0   
8819376            16.916667   22.0         519  20.0          4    9.0     0   
8819377            17.666667   22.0         519  21.0          4    9.0     0   
8819378            18.333333   22.0         519  22.0          4    9.0     0   

         Rain  
0          

In [8]:
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

# Cargar los datos en lotes y realizar transformaciones
# Cargar los datos en lotes y realizar transformaciones
def load_and_process_data(csv_file):
  data=csv_file
    # Cargar el lote de datos desde el archivo CSV
  data.dropna(subset=['num_docks_available', 'Llocs','station_id','hour','dayofweek','month','Wind','Rain'], inplace=True)
  data['porcio']=(data['num_docks_available']/data['Llocs'])
  data['festa']=0
  data['festa']=data['festa'].where(data['dayofweek']>5,1)
  data['festa']=data['festa'].where(data['month']==8,1)
  data['winter']=0
  data['winter']=data['winter'].where(data['month']==12,1)
  data['winter']=data['winter'].where(data['month']==1,1)
  data['winter']=data['winter'].where(data['month']==2,1)


  data['spring']=0
  data['spring']=data['spring'].where(data['month']==3,1)
  data['spring']=data['spring'].where(data['month']==4,1)
  data['spring']=data['spring'].where(data['month']==5,1)
  data['summer']=0
  data['summer']=data['summer'].where(data['month']==6,1)
  data['summer']=data['summer'].where(data['month']==7,1)
  data['summer']=data['summer'].where(data['month']==8,1)

  data['fall']=0
  data['fall']=data['fall'].where(data['month']==9,1)
  data['fall']=data['fall'].where(data['month']==10,1)
  data['fall']=data['fall'].where(data['month']==11,1)
  data.dropna(subset=['porcio'], inplace=True)
    # Realizar las transformaciones necesarias en los datos
    # Por ejemplo, codificar variables categóricas, escalar valores numéricos, etc.

    # Dividir los datos en características (X) y etiquetas (Y)
  X = data[['station_id','hour','festa','month','Rain']]
  X = data[['station_id','hour','festa','winter','spring','summer','fall','Rain']]
    # X = data[['station_id','hour','dayofweek','Rain_Lectura']]

  Y = (data['porcio'])

    # Devolver los datos procesados
  return X, Y
def load_and_process_data2(csv_file):
  data = pd.read_csv(csv_file)
    # Cargar el lote de datos desde el archivo CSV
    # Realizar las transformaciones necesarias en los datos
    # Por ejemplo, codificar variables categóricas, escalar valores numéricos, etc.
  data['festa']=0
  data['festa']=data['festa'].where(data['dayofweek']>5,1)
  data['festa']=data['festa'].where(data['month']==8,1)
  data['winter']=0
  data['winter']=data['winter'].where(data['month']==12,1)
  data['winter']=data['winter'].where(data['month']==1,1)
  data['winter']=data['winter'].where(data['month']==2,1)


  data['spring']=0
  data['spring']=data['spring'].where(data['month']==3,1)
  data['spring']=data['spring'].where(data['month']==4,1)
  data['spring']=data['spring'].where(data['month']==5,1)
  data['summer']=0
  data['summer']=data['summer'].where(data['month']==6,1)
  data['summer']=data['summer'].where(data['month']==7,1)
  data['summer']=data['summer'].where(data['month']==8,1)

  data['fall']=0
  data['fall']=data['fall'].where(data['month']==9,1)
  data['fall']=data['fall'].where(data['month']==10,1)
  data['fall']=data['fall'].where(data['month']==11,1)
    # Dividir los datos en características (X) y etiquetas (Y)
  X = data[['station_id','hour','festa','month','Rain']]
  X = data[['station_id','hour','festa','winter','spring','summer','fall','Rain']]

    # Devolver los datos procesados
  return X

X_train, Y_train = load_and_process_data(dataframe_final)

# Escalar los valores numéricos en el rango [0, 1]
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Crear un modelo de red neuronal
model = tf.keras.Sequential([
    tf.keras.layers.Dense(256, activation='relu', input_shape=(8,)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1)
])

# Definir el optimizador Adam con una tasa de aprendizaje variable
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.001,
    decay_steps=1000,
    decay_rate=0.9
)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

# Compilar el modelo
model.compile(optimizer=optimizer, loss='mean_squared_error')

# Entrenar el modelo en los conjuntos de entrenamiento
model.fit(X_train_scaled, Y_train, epochs=10, batch_size=32)

# Realizar predicciones con el modelo entrenado
new_data = load_and_process_data2('/content/drive/MyDrive/prova/test/submission.csv')
new_data_scaled = scaler.transform(new_data)

predictions = model.predict(new_data_scaled)
ids = new_data.index.values

# Crear un DataFrame con las predicciones y los IDs
df = pd.DataFrame({'index': ids, 'percentage_docks_available': predictions[:, 0]})

# Definir la ruta y el nombre del archivo CSV
csv_file = 'predictions.csv'

# Guardar el DataFrame en el archivo CSV
df.to_csv(csv_file, index=False)
files.download('/content/'+csv_file)


Epoch 1/10
275555/275555 [==============================] - 708s 3ms/step - loss: 0.0779
Epoch 2/10
275555/275555 [==============================] - 712s 3ms/step - loss: 0.0766
Epoch 3/10
275555/275555 [==============================] - 703s 3ms/step - loss: 0.0766
Epoch 4/10
275555/275555 [==============================] - 703s 3ms/step - loss: 0.0766
Epoch 5/10
275555/275555 [==============================] - 704s 3ms/step - loss: 0.0766
Epoch 6/10
275555/275555 [==============================] - 722s 3ms/step - loss: 0.0766
Epoch 7/10
275555/275555 [==============================] - 727s 3ms/step - loss: 0.0766
Epoch 8/10
275555/275555 [==============================] - 737s 3ms/step - loss: 0.0766
Epoch 9/10
275555/275555 [==============================] - 730s 3ms/step - loss: 0.0766
Epoch 10/10
1719/1719 [==============================] - 2s 1ms/step


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>